In [10]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [11]:
from helpers import load_csv_data

DATA_TRAIN_PATH = 'data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [12]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    relu_arg = y * X.dot(w)
    zeros = np.zeros(len(relu_arg))
    relu_term  = np.sum(np.max([zeros,1-relu_arg], axis = 0))
    pen_term = 0.5 * lambda_ * w.dot(w)
    return relu_term + pen_term

In [13]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    labels = y
    prediction = (X.dot(w)>0)*2 - 1 #cast 1, 0 into 1, -1
    accuracy = np.mean(prediction==labels)
    return accuracy

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [14]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    
    x_n, y_n = X[n], y[n]
    z_n = y_n*x_n.dot(w)
    dl_n = -int(z_n<1)
    return dl_n * y_n * x_n + lambda_ * w / num_examples

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [15]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=18129815729.16374
iteration=10000, cost=20030166.74043373
iteration=20000, cost=16407386.150163058
iteration=30000, cost=14576354.898443233
iteration=40000, cost=13454599.898593813
iteration=50000, cost=12441619.592382537
iteration=60000, cost=11925257.175384462
iteration=70000, cost=11239156.779409673
iteration=80000, cost=10663546.951377464
iteration=90000, cost=10274106.856531827
training accuracy = 0.6956


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [7]:

def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    # Compute projected gradient
    # We take the negative gradient since we seek to maximize G(alpha, w).
    gradient = - (1 - y_n * x_n.dot(w))
    # Compute projected gradient.
    if old_alpha_n == 0:
        projected_gradient = min(0, gradient)
    elif old_alpha_n == 1:
        projected_gradient = max(0, gradient)
    else: # if alpha_n in (0,1)
        projected_gradient = gradient
    # Check if an update is necessary:
    if projected_gradient == 0:
        alpha[n] = old_alpha_n
    else:
        # If necessary, update alpha
        raw_new_alpha_n = old_alpha_n - lambda_ * gradient/x_n.dot(x_n) 
        # Project onto domain.
        alpha[n] = min(1, max(0, raw_new_alpha_n))
    #w = X.T.dot(np.diag(y).dot(alpha))/lambda_ #is a slow update.
    w += (alpha[n] - old_alpha_n) * y_n *x_n / lambda_ #This update is faster.
    return w, alpha

In [8]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    G = sum(alpha * (1 - y * X.dot(w))) + lambda_ * w.dot(w) / 2
    return G

In [9]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))
    

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:3555.87917, dual:0.00000, gap:3555.87917
iteration=10000, primal:4211.29209, dual:0.00010, gap:4211.29199
iteration=20000, primal:5775.30423, dual:0.00019, gap:5775.30404
iteration=30000, primal:4081.94891, dual:0.00027, gap:4081.94864
iteration=40000, primal:3234.31699, dual:0.00036, gap:3234.31663
iteration=50000, primal:3417.03458, dual:0.00045, gap:3417.03412
iteration=60000, primal:4466.90349, dual:0.00054, gap:4466.90295
iteration=70000, primal:4340.61308, dual:0.00062, gap:4340.61246
iteration=80000, primal:3478.83483, dual:0.00071, gap:3478.83412
iteration=90000, primal:3183.65491, dual:0.00080, gap:3183.65411
training accuracy = 0.5114
